In [1]:

# importing the requests library
import requests
import pandas as  pd
import pickle
import re

In [2]:
data = pd.read_csv('./silver_dataset.csv')

# tweets = data['tweet'].tolist()

In [3]:
# api-endpoint
ENDPOINT = "https://sentic.net/api/en/"

In [4]:
POLARITY_KEY = 'UBHiejvp0lXkG.py'

In [5]:
URL = ENDPOINT + POLARITY_KEY

In [6]:
# location given here
text = "I've had my 2nd dose of Covid :Vaccine. I got myself vaccinated to protect my service , my colleagues and myself. As a doctor I am convinced of its safety and efficacy and would encourage everyone else to take up the opportunity to get the vaccine too when it is offered"
  
# defining a params dict for the parameters to be sent to the API
PARAMS = {'text':text}
  
# sending get request and saving the response as response object
r = requests.get(url = URL, params = PARAMS)
print(r.text)

POSITIVE



In [7]:
import asyncio
import aiohttp

In [8]:
outputs = []
cnt = 0

def preprocess(text):
    for c in [';', '&', '#']: 
        text = text.replace(c, ':')
    text = re.sub('<EMOJI[\S]*>', '', text)
    text = re.sub('<USER>', '', text)
    return text

async def get(session, text, id):
    try:
        async with session.get(url=URL, params = {'text':preprocess(text)}) as response:
            resp = await response.read()
            return [id, resp]
    except Exception as e:
        print("Unable to get url due to {}.".format(e.__class__))


async def main(data):
    async with aiohttp.ClientSession() as session:
        ret = await asyncio.gather(*[get(session, row['tweet'], row['id']) for idx, row in data.iterrows()])
    print("Finalized all. Return is a list of len {} outputs.".format(len(ret)))
    # print(ret)
    outputs.extend(ret)

# l = len(tweets)
loop = asyncio.get_event_loop()
loop.run_until_complete(main(data))
# fl = open('outputs', 'wb')
# pickle.dump(outputs, fl)

Finalized all. Return is a list of len 14348 outputs.


In [9]:
fl = open('outputs', 'wb')
pickle.dump(outputs, fl)